Ce notebook sert à tester la classe Evaluator avec des couts initiaux. Les performances devraient être de l'ordre de 88% de bonne classification en 90/10

In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [2]:
import os
dataset_path = os.getenv('MAO_DATASET_PATH')


In [3]:
from evaluation.evaluate import Evaluator
from gklearn.utils.graphfiles import loadDataset
import torch
import numpy as np

In [4]:
Gs,y = loadDataset(dataset_path)

In [5]:
evaluator  = Evaluator(Gs)

['C', 'N', 'O']
{'C': 0, 'N': 1, 'O': 2} 3
3
torch.Size([68, 729])
adjacency matrices tensor([[        0,         1,         0,  ...,         0,         0,
         524055856],
        [        0,         1,         0,  ...,         0, 567927792,
             32724],
        [        0,         1,         0,  ...,         0,         0,
                 0],
        ...,
        [        0,         1,         0,  ...,         0,         0,
                 0],
        [        0,         1,         0,  ...,         0,         0,
                 0],
        [        0,         1,         0,  ...,         0,         0,
                 0]], dtype=torch.int32)
node labels tensor([[         1,          0,          0,  ..., 1645530528,      32723,
         1645505104],
        [         1,          0,          0,  ...,      32723, 1645128880,
              32723],
        [         1,          0,          0,  ..., 1644544528,      32723,
         1645006640],
        ...,
        [         1

In [17]:
# Création des poids initiaux, sans être appris
nb_node_labels = evaluator.nb_labels
cns = torch.ones(nb_node_labels,nb_node_labels) 
cns = cns - torch.diag(cns.diag())
print(cns)

nb_edge_labels = evaluator.nb_edge_labels
ces = torch.ones(nb_edge_labels,nb_edge_labels) 
ces = ces - torch.diag(ces.diag())
print(ces)

cndl = torch.ones(1) * 3
cedl = torch.ones(1) * 3

print(cndl,cedl)

tensor([[0., 1., 1.],
        [1., 0., 1.],
        [1., 1., 0.]])
tensor([[0., 1., 1.],
        [1., 0., 1.],
        [1., 1., 0.]])
tensor([3.]) tensor([3.])


In [18]:
nb_graph_dataset = len(Gs)
shuffled_indexes = np.random.permutation(range(nb_graph_dataset))
nb_test_data = nb_graph_dataset//10
nb_train_data = nb_graph_dataset - nb_test_data

train_data = shuffled_indexes[:nb_train_data]
test_data = shuffled_indexes[nb_train_data:]
print(train_data,test_data)

y_train = [y[train_index] for train_index in train_data]
y_test = [y[test_index] for test_index in test_data]
print(y_train, y_test)

[61 14 41 37 35 30 64 10 32 31 52 49 34  8 12 66 28 40  2 46  0 38  9 23
 39 33 11  3 50  4 48 20 21 44 45 36 19  5 47 51 15 43 25 27 63  7 67 18
 60 13 59 55 56 42 58 26 29 16 62 65 24  6] [22 53 54  1 17 57]
[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0] [1.0, 1.0, 0.0, 0.0, 1.0, 0.0]


In [21]:
k=3
D_train, D_test = evaluator.classification(train_data, test_data, y_train,y_test,k,cns,ces,cndl,cedl)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:16<00:00,  2.82s/it]


Test Results : 
              precision    recall  f1-score   support

         0.0       1.00      0.67      0.80         3
         1.0       0.75      1.00      0.86         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6

[[2 1]
 [0 3]]
